# 1.Motivation

Initially we wanted to use what we have learned so far with the SF dataset and apply those and more methods to a lesser known criminal dataset. We sought after getting our hands on a dataset from Iceland, but due to the fact that not many people live in Iceland and it would be easy to see who committed the crime as everyone basically knows everyone. Therefore the contact person we had at the office of the National Commissioner of the Icelandic Police  raised some privacy concerns and eventually denied us of the opportunity. Having literally no time to find another good idea, we stumbled upon the NYC open dataset for New York City Seven Major felony incidents. That dataset supposedly had criminal data dating back to the year 1900, and we were excited to think that we could possibly visualize the growth or decline of crimes in New York City amongst doing similar actions as we had done with the San Fransisco dataset. So we transferred our initial aim from Iceland to New York. What we want the end user to be able to do is visualize the following :

* Recognize what crime is least or most likely to occur or in a position they choose on a map. Is there a location one should avoid?		
* Heatmap of at what time a crime is likely to be made and at which day of the week.
* See on a map the offenses committed at every year, are there any crime types that are growing.
* See the center positioning (K-Means centroids) of every offense on a map, thereby possibly figure out where a police station would be ideally located in terms of offenses.
* Are crimes higher than average in the five boroughs, looking at the mean statistic in a barchart.
* Visualize in a heatmap what part of the year and or month has the most crime occurrences.
* Visualize the evolution of crime frequency in NYC (ended up not being possible)


# 2. Basic Stats. Lets understand the dataset better


As the dataset has offense occurrences that date back to the year 1905 there was bound to be some occurrences that had missing details. Majority of the data before the year 2007 especially seems to be added afterwards. Therefore before we did our decision tree and K-Means centroids we exluded all the occurrences that did not contain values for the *Precinct,Offense, Day of Week and Occurrence Hour*. Also there are a total of 26 miscellaneous jurisdiction’s in the dataset, we excluded all of them except fot the *N.Y. Police Dept.* Before this cleaning we had 1.123.465 row values, after cleaning we had 1.040.409 row values, so we had skimmed roughly 7% of the data. However as we do a lot of the visualizations working directly with the NYC SODA API we do not do any datacleaning or preprocessing on the dataset for them.

The dataset itself is approx 200MB of single text stored in a .csv format, each row with 20 different variables. The dataset’s oldest record is from april 1st 1900 and the most recent one is up until december 31st 2015.


# 3.Theory. Which theoretical tools did you use? 

To be able to predict what crime could take place at a given position we used the SKLearn DecisionTree classifier, as we have used a similar approach in a dataset before we felt this was the obvious tool to use.Decision Tree’s create a model that predict the value of a target variable by learning simple decision rules inferred from the data features [1]. We used the SKLearn Cross-Validation Train_Test_Split [3] module to split the data into test and training data. We ended up chosing to have have the test size at 10% as 20% did not have that much of an effect , using the *precinct* data as our input data (X) and *offenses* as our prediction values (y). The model works as expected, mainly because there are only seven major felony’s in the dataset versus 78 precincts. We measured our model by looking at the mean accuracy score which was about 0.38. As a comparision we looked at reversing the model, that is having the *offenses* as our input data (X) and *precincts* as our prediction values (y), as a result we got the mean accuracy score of 0.04. The results from this classification are available dynamic from the web service like most of the visualization on the site. We trained and fitted the classifier in Jupyter Notebook, exported the model to a joblib object (which is very similar to Python pickle) which we place on our Heroku service. Then D3 request the service on each click on the precinct map.

For us to be able to locate the center point K-Means (centroid) of a crime cluster we again use the approach we were taught on the SF dataset earlier, by using the K-Means Cluster module. We however only used offense data from the year 2015 to limit the data somewhat. The K-means algorithm aims to choose centroids that minimise the *inertia*, or within-cluster sum of squared criterion [2]. We ended up using the same default number of clusters as we did in the aforementioned SF dataset, 2,3,4,5 and 6 number of clusters for the end user to see visually the effect the various cluster number sizes has on our data. Also we measured the score each cluster size had for each of the offenses, were we came to the conclusion that the higher the amount of offense occurrences the more the cluster size mattered. For instance for the *Murder & Non-Negl. Manslaughter* there were 336 offenses in 2015 and the K-Means score varied from -2.14( clusterSize = 2 )  to -0.51(clusterSize = 6), whereas for the *Grand Larceny* offense there were 41873 offenses in 2015 and the K-Means score varied from -279.37 ( clusterSize = 2) to -75.99 (clusterSize = 6). This we thought was to be expected. 


#### Decision Tree Notebook Link

https://github.com/finkol/social_final_2016/blob/master/notebooks/Prediction%20Decision%20Tree%20Code.ipynb

#### K-Means Notebook Link

https://github.com/finkol/social_final_2016/blob/master/notebooks/K-Means%20Clustering%20Centroid%20Code%20.ipynb

# 4.Visualizations

To start with we visualize the number of felonies by offense type with a regular barchart, nothing fancy but just to get the idea clearly what felonies occur most often, the higher the bar the more occurrences there have been of course. Then again in another barchart the user can select a borough to see the ratio of those top felonies and compare them with the average ratio of all the boroughs combined. Using the barchart for these basic data representations we feel works well to tell that initial story of our data.

For the user to be able to get an idea of the growth of felony offenses by each year, and the location of those felonies in NYC we chose to visualize it in a dotmap. For each year a dot is generated on the map of NYC representing an occurrence of that crime, when looking at more recent year the dots become quite densed. Using the year slider and those dots, we feel is a far better visualization than a barchart. Which would probably be to big.

We want the user to see at what time of the week a crime is most likely to occur and by visualizing that in a heat diagram, where the darker the color is the more crimes have happened at that time should paint a clear picture for the user.  
We use that heat diagram again to visualize all crime occurrences over the span of a year, which helps to see at what in a month or month in a year a crime is more likley to occur.

As we are using K-Means clustering centroids to be able to pinpoint the center location of a felony on a map, the best way to tell that story is undoubtfully by placing that centroid on a map.

In the Decicison tree prediction, the user is able to click all the precinct on the map and the site will display which offense is most likely there. 67 of 78 precincts have Grand Larceny as the most likely offense.

Most of the visualzations are served right from our Python Flask server where our service call the SODA API[7] for the convenience of not having to parse the CSV file. The SODA API uses Socrata Query Language (SoQL) which is similar to SQL and very interesting to learn. It was much quicker than having to have 200 MB CSV file in memory. It would also not be possible have that big CSV on Heroku to handle the requests from D3.


#### The file that handled the requests can be seen here:
https://github.com/finkol/social_final_2016/blob/master/social_final.py

#### The files with the logic and calls to the SODA API are in this folder:
https://github.com/finkol/social_final_2016/tree/master/services

# 5.Discussion.

What worried us when starting to work on the dataset was that the data prior to 2007 was really insufficient. We had set out to see if the crime rate had really dropped in the new millenium in comparision to for example the 1980’s or 90’s, were crimerate was supposedly at an all time high in NYC because of a so called *crack epidemic* [4]. Because of this poor data our main research approach was faulted. We also got a suggestion to use postal zip codes to identify areas instead of using the NYC boroughs, our dataset did not have the zip code information. It does however have coordinate location info for the crimes, but in order for us to get the zip codes for those coordinates we both tried the *Google Maps API* [5] and the *Geopy* [6] python package to fetch every zip code. However the dataset being more than million lines and both those services having daily limits (google 2500 request limit per day), we unfortunately weren’t able to use zipcodes after having used a couple of days to figure out a way to do it. We therefore decided to use the police precincts as the dataset has that information. 

Although stumbling upon the fact that data collection in the last century was insufficient and not being able to use zip codes as was suggested to us. The precincts usage served its purpose and with the help of D3.js we could find new and fun ways to visualize crime offense patterns in the dataset. 

# 6. References

1. http://scikit-learn.org/stable/modules/tree.html

2. http://scikit-learn.org/stable/modules/clustering.html

3. http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.train_test_split.html

4. https://en.wikipedia.org/wiki/Crack_epidemic

5. https://developers.google.com/maps/

6. https://pypi.python.org/pypi/geopy

7. https://dev.socrata.com/
